In [ ]:
! pip install transformers datasets accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.1 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import transformers
import pandas as pd
from datasets import load_dataset
import evaluate # load_metric from datasets is removed
import numpy as np
import random
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import set_seed
import torch

In [ ]:
# loading a dataset from the csvs at https://github.com/inflaton/Explainable-Sentiment-Analysis-with-DeepSeek-R1/tree/main/dataset
dataset = load_dataset(
    "csv",
    data_files={
        "train": "GoEmotions-train.csv",
        "validation": "GoEmotions-val.csv",
        "test": "GoEmotions-test.csv",
    }
)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
precision = evaluate.load('precision')
recall = evaluate.load('recall')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    acc = accuracy.compute(predictions=predictions, references=labels)

    f1_macro = f1.compute(
        predictions=predictions,
        references=labels,
        average="macro"
    )

    f1_weighted = f1.compute(
        predictions=predictions,
        references=labels,
        average="weighted"
    )

    recall_macro = recall.compute(
        predictions=predictions,
        references=labels,
        average="macro"
    )
    precision_macro = precision.compute(
        predictions=predictions,
        references=labels,
        average="macro"
    )

    recall_weighted = recall.compute(
        predictions=predictions,
        references=labels,
        average="weighted"
    )
    precision_weighted = precision.compute(
        predictions=predictions,
        references=labels,
        average="weighted"
    )

    return {
        "accuracy": acc["accuracy"],
        "f1_macro": f1_macro["f1"],
        "f1_weighted": f1_weighted["f1"],
        "recall_macro": recall_macro["recall"],
        "precision_macro": precision_macro["precision"],
        "recall_weighted": recall_weighted["recall"],
        "precision_weighted": precision_weighted["precision"]
    }

In [ ]:
bert_large = AutoModelForSequenceClassification.from_pretrained("virtual12345/bert-large-uncased-finetuned-GoEmotions")
bert_base = AutoModelForSequenceClassification.from_pretrained("virtual12345/bert-base-uncased-finetuned-GoEmotions")
distilbert_base = AutoModelForSequenceClassification.from_pretrained("virtual12345/distilbert-base-uncased-finetuned-GoEmotions")

config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
# Label maps
labels = sorted(list(set(dataset["train"]["Emotion"])))
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}

# Encode string labels to ints
def encode_labels(example):
    example["labels"] = label2id[example["Emotion"]]
    return example

dataset = dataset.map(encode_labels)
test_data = dataset["test"]

# tokenization for each model
def make_preprocess(tokenizer):
    def preprocess(examples):
        return tokenizer(
            examples["Text"], # assuming dataset has a column named "Text", which is true in this case
            truncation=True
        )
    return preprocess


Map:   0%|          | 0/23485 [00:00<?, ? examples/s]

Map:   0%|          | 0/2956 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [ ]:
def evaluate_model(model, eval_batch_size):
    full_name = model.config._name_or_path
    tokenizer = AutoTokenizer.from_pretrained(full_name, use_fast=True)
    preprocess_model = make_preprocess(tokenizer = tokenizer)
    encoded_dataset = test_data.map(preprocess_model, batched=True)

    model_name = full_name.split("/")[-1] # the model name, not including the username

    args = TrainingArguments(
        output_dir= f"{model_name}_results",
        per_device_eval_batch_size=eval_batch_size,
        fp16=True,
        push_to_hub=True,
        hub_private_repo=True,
        hub_model_id=full_name, # push to my model's existing repo
        report_to = 'none', # avoid annoying wandb prompt
        dataloader_num_workers=2,
    )

    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=args,
        compute_metrics=compute_metrics, # my metrics function
    )

    metrics = trainer.evaluate(encoded_dataset)
    metrics = {k: float(v) for k, v in metrics.items()}

    trainer.log_metrics("test", metrics)
    trainer.save_metrics("test", metrics)

    trainer.push_to_hub() # making SURE it pushes to hub
    return metrics


In [ ]:
bert_large_results = evaluate_model(bert_large, eval_batch_size = 64)
bert_base_results = evaluate_model(bert_base, eval_batch_size = 128)
distilbert_base_results = evaluate_model(distilbert_base, eval_batch_size = 256)

/tmp/ipython-input-7274829.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


***** test metrics *****
  eval_accuracy               =     0.6317
  eval_f1_macro               =     0.5674
  eval_f1_weighted            =     0.6235
  eval_loss                   =     1.3138
  eval_model_preparation_time =     0.0057
  eval_precision_macro        =      0.567
  eval_precision_weighted     =     0.6283
  eval_recall_macro           =      0.585
  eval_recall_weighted        =     0.6317
  eval_runtime                = 0:00:00.64
  eval_samples_per_second     =     934.04
  eval_steps_per_second       =     15.567


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

...

***** test metrics *****
  eval_accuracy               =      0.605
  eval_f1_macro               =     0.4961
  eval_f1_weighted            =     0.5962
  eval_loss                   =     1.4305
  eval_model_preparation_time =     0.0016
  eval_precision_macro        =     0.5035
  eval_precision_weighted     =     0.5976
  eval_recall_macro           =     0.5016
  eval_recall_weighted        =      0.605
  eval_runtime                = 0:00:00.29
  eval_samples_per_second     =   2006.239
  eval_steps_per_second       =     10.031


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...results/model.safetensors:   9%|9         | 25.1MB /  268MB            

  ...results/training_args.bin: 100%|##########| 5.97kB / 5.97kB            

In [ ]:
print(bert_large_results)
print(bert_base_results)
print(distilbert_base_results)

{'eval_loss': 1.3137543201446533, 'eval_model_preparation_time': 0.0057, 'eval_accuracy': 0.6316666666666667, 'eval_f1_macro': 0.5674422537922372, 'eval_f1_weighted': 0.6234580724976357, 'eval_recall_macro': 0.5850316255470654, 'eval_precision_macro': 0.5669915790108452, 'eval_recall_weighted': 0.6316666666666667, 'eval_precision_weighted': 0.6283416850536854, 'eval_runtime': 0.6424, 'eval_samples_per_second': 934.04, 'eval_steps_per_second': 15.567}
{'eval_loss': 1.3980751037597656, 'eval_model_preparation_time': 0.0031, 'eval_accuracy': 0.6033333333333334, 'eval_f1_macro': 0.5215751874142776, 'eval_f1_weighted': 0.598739774870056, 'eval_recall_macro': 0.5213643821555971, 'eval_precision_macro': 0.5493188054772145, 'eval_recall_weighted': 0.6033333333333334, 'eval_precision_weighted': 0.6073548150848161, 'eval_runtime': 0.329, 'eval_samples_per_second': 1823.769, 'eval_steps_per_second': 15.198}
{'eval_loss': 1.4305089712142944, 'eval_model_preparation_time': 0.0016, 'eval_accuracy': 